### Video Game Sales Forecast
___
The goal of this project is to uncover advertising strategies for a global video game retailer. We have historical video game sales data for many platforms and areas of the world.

In this notebook we will load and explore the data developing a plan of action to prepare this data for exploration.

#### Pre-processing data
___
Step 1:<br>
    - Load data and take a look at what we are working with.<br>
    - Personalize Pandas settings<br>
    <br>
Step 2:<br>
    - Change column names to lowercase convention, removing spaces if needed<br>
    - Deal with duplicates, complete and partial, then drop them<br>
    <br>
Step 3:<br>
    - Handle missing values<br>
    - How many, distribution across columns/ rows<br>
    - Drop rows with too many missing values<br>
    - Fill missing values with placeholders<br>
    - Replace/ update current placeholder missing values<br>
    <br>
Step 4:<br>
    - Standardize spelling conventions, making them lowercase, updating values as needed<br>
    - Ratings, genre, name columns<br>
    - Make critic and user score the same range<br>
    <br>
Step 5:<br>
    - Feature engineering, create more data based on current values<br>
    - Create columns: total/ global sales, brand, mobile device<br>
    <br>
Step 6:<br>
    - Store processed data for Exploratory Data Analysis


In [1]:
# load pandas
import pandas as pd

In [2]:
# store data
game = pd.read_csv('../games.csv')

In [3]:
# take a look at our data
game.info()
game.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


Delveloping a plan:
From the first glance our data is going to need at least several adjustments. 

- column names turned into lower case
- missing values, sometimes a lot of them
- appears that the name column has the longest data entry (physical length)

In [ ]:
# what is the longest name string? for pandas customization
game['Name'].str.len().max()

132.0

In [ ]:
# personalize/modify pandas

# work with max rows == None but it makes scrolling annoying
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
pd.set_option('display.max_colwidth', 132)

In [6]:
# updating column spellings to lower case

# create a list of column names
col_names = list(game.columns)

# convert column names to lower case, and overwrite the column names
game.columns = list(map(lambda x: x.lower(), col_names))

#### Duplicates
___
Looking into duplicates. Search for complete duplicates, and paired unique value duplicates as well. Removing any duplicates we find.

In [7]:
# complete duplicates
game.duplicated().sum()

0

In [8]:
# zero complete duplicates, how about
# paired duplicates between name and platform

# sear
test = game.duplicated(subset=['name', 'platform'], keep='last').reset_index()
test = test[test[0] == True]
dup_index = list(test['index'])
dup_index

[604, 659, 1190, 1745, 5972]

In [9]:
test = game.duplicated(subset=['name', 'platform'], keep='first').reset_index()
test = test[test[0] == True]
dup_index2 = list(test['index'])
dup_index2

[1591, 4127, 11715, 14244, 16230]

In [10]:
# remove them
game = game.drop(dup_index)

In [11]:
# new version
game.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16710 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16709 non-null  object 
 1   platform         16710 non-null  object 
 2   year_of_release  16441 non-null  float64
 3   genre            16709 non-null  object 
 4   na_sales         16710 non-null  float64
 5   eu_sales         16710 non-null  float64
 6   jp_sales         16710 non-null  float64
 7   other_sales      16710 non-null  float64
 8   critic_score     8133 non-null   float64
 9   user_score       10010 non-null  object 
 10  rating           9945 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.5+ MB


#### Global missing values
___
Look into the missing values.<br>
How is the distribution? <br>
Are there areas with too much missing info to be useable?<br>
Fill missing values as needed, the goal is preserve as much data as possible<br>

In [12]:
# to what extent are rows missing values?
# find out how much of our data is missing a lot of values

# list for storage
totals = []

# loop through possible maximum missing values (column count)
for threshold in range(11):

    # create a dictionary:
    # key = threshold for missing values in a row
    # values = number of rows missing the threshold amount of data
    temp_dict = {threshold: len(game[game.isna().sum(axis=1) >= threshold]),
                 "% of total rows": len(game[game.isna().sum(axis=1) >= threshold])/len(game)}

    # store them
    totals.append(temp_dict)

# show them
print(totals)

[{0: 16710, '% of total rows': 1.0}, {1: 8836, '% of total rows': 0.528785158587672}, {2: 6765, '% of total rows': 0.40484739676840215}, {3: 6625, '% of total rows': 0.3964691801316577}, {4: 86, '% of total rows': 0.005146618791143028}, {5: 1, '% of total rows': 5.984440454817474e-05}, {6: 0, '% of total rows': 0.0}, {7: 0, '% of total rows': 0.0}, {8: 0, '% of total rows': 0.0}, {9: 0, '% of total rows': 0.0}, {10: 0, '% of total rows': 0.0}]


We knew there was quite a lot of missing values, but we weren't certain how they were distributed across the dataframe. ~40% have 3 missing values, but only 0.5% have 4 missing. That's a clear cut off.

In [13]:
# which rows have 4 missing values?

# filter for rows with at least 4 missing values
df_filtered = game[game.isna().sum(axis=1) >= 4]

df_filtered

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
475,wwe Smackdown vs. Raw 2006,PS2,NaN,Fighting,1.57,1.02,0.00,0.41,NaN,NaN,NaN
609,Space Invaders,2600,NaN,Shooter,2.36,0.14,0.00,0.03,NaN,NaN,NaN
1301,Triple Play 99,PS,NaN,Sports,0.81,0.55,0.00,0.10,NaN,NaN,NaN
1506,Adventure,2600,NaN,Adventure,1.21,0.08,0.00,0.01,NaN,NaN,NaN
1585,Combat,2600,NaN,Action,1.17,0.07,0.00,0.01,NaN,NaN,NaN
2010,Donkey Kong Land III,GB,NaN,Platform,0.68,0.31,0.00,0.04,NaN,NaN,NaN
2106,Air-Sea Battle,2600,NaN,Shooter,0.91,0.06,0.00,0.01,NaN,NaN,NaN
2361,Rhythm Heaven,Wii,NaN,Misc,0.11,0.00,0.77,0.01,NaN,NaN,NaN
2572,PES 2009: Pro Evolution Soccer,PSP,NaN,Sports,0.04,0.33,0.26,0.17,NaN,NaN,NaN
2773,WarioWare: Twisted!,GBA,NaN,Puzzle,0.16,0.06,0.50,0.02,NaN,NaN,NaN


thoughts
___
All of these are missing the score/ ratings (3 columns) and then the release year (except the weird no names)

This data actually isn't that bad, so we won't simply drop it. (maybe? I don't know how I will utilize the ratings yet)

Other insight: if the video game had any meaningful sales it was on the atari 2600. Atari data might get dropped because we won't be advertising for super old video games, and the sales numbers are adjusted for inflation? If they are adjusted then they are quite low. If they aren't adjusted they won't be able to relate to modern sales figures. 

That will get settled later.

For now let's get rid of the columns missing 5 values. 

In [14]:
# drop rows missing at least 5 values
game = game[game.isna().sum(axis=1) < 5]

#### Spelling conventions
___
Missing values have been removed, time to convert to lowercase, remove spaces if needed, standardize column values, rename video game platforms

In [15]:
# rating column still has a number of missing values
game['rating'].value_counts(dropna=False)

rating
NaN     6764
E       3989
T       2959
M       1563
E10+    1419
EC         8
K-A        3
RP         3
AO         1
Name: count, dtype: int64

In [16]:
# update the values in the rating column while handling the missing values

# fill na's with na
game['rating'] = game['rating'].fillna('na')

# build a dictionary to rename rating column
name_dict = {'E': 'everyone',
             'T': 'teen',
             'M': 'mature',
             'E10+': 'everyone_over_ten',
             'EC': 'early childhood',
             'K-A': 'everyone',
             'RP': 'rating_pending',
             'AO': 'adults only',
             }

# replace the old values with new values
game['rating'] = game['rating'].replace(name_dict)

In [17]:
# convert the values to lowercase

# create a list of 'name' column
col_value = list(game['name'])

# replace old name with new name
game.name = list(map(lambda x: x.lower(), col_value))

# 'platform' column
col_value = list(game.platform)
game.platform = list(map(lambda x: x.lower(), col_value))

# 'genre' column
col_value = list(game.genre)
game.genre = list(map(lambda x: x.lower(), col_value))

In [18]:
# now that strings are lowercase, let's run the duplicate pairs one more time

# from earlier
# paired duplicates between name and platform
test = game.duplicated(subset=['name', 'platform'], keep='last').reset_index()
test = test[test[0] == True]
dup_index = list(test['index'])
dup_index

[]

In [19]:
# reformat the naming convention for

# build a dictionary to rename platforms
platform_dict = {'pcfx': 'pc-fx',
                 'gg': 'game_gear',
                 'tg16': 'turbo_grafx-16',
                 '3do': '3do',
                 'ws': 'wonderswan',
                 'scd': 'sega_cd',
                 'ng': 'neo_geo',
                 'gen': 'genesis',
                 'dc': 'dreamcast',
                 'gb': 'gameboy',
                 'nes': 'nintendo_entertainment_system',
                 '2600': '2600',
                 'wiiu': 'wii_u',
                 'sat': 'saturn',
                 'snes': 'super_nintendo_entertainment_system',
                 'xone': 'xbox_one',
                 'n64': 'nintendo_64',
                 'ps4': 'playstation_4',
                 'psv': 'playstation_vita',
                 '3ds': '3ds',
                 'gc': 'game_cube',
                 'gba': 'gameboy_advanced',
                 'xb': 'xbox',
                 'pc': 'computer',
                 'ps': 'playstation',
                 'psp': 'playstation_portable',
                 'x360': 'xbox_360',
                 'wii': 'wii',
                 'ps3': 'playstation_3',
                 'ds': 'ds',
                 'ps2': 'playstation_2',
                 }

# replace the old values with new values
game['platform'] = game['platform'].replace(platform_dict)

In [20]:
# create a brand column

# build a dictionary to map from the platform column
brand_dict = {'pc-fx': 'nec',
              'game_gear': 'sega',
              'turbo_grafx-16': 'nec',
              '3do': 'panasonic',
              'wonderswan': 'bandai',
              'sega_cd': 'sega',
              'neo_geo': 'snk',
              'genesis': 'sega',
              'dreamcast': 'sega',
              'gameboy': 'nintendo',
              'nintendo_entertainment_system': 'nintendo',
              '2600': 'atari',
              'wii_u': 'nintendo',
              'saturn': 'sega',
              'super_nintendo_entertainment_system': 'nintendo',
              'xbox_one': 'microsoft',
              'nintendo_64': 'nintendo',
              'playstation_4': 'sony',
              'playstation_vita': 'sony',
              '3ds': 'nintendo',
              'game_cube': 'nintendo',
              'gameboy_advanced': 'nintendo',
              'xbox': 'microsoft',
              'computer': 'computer',
              'playstation': 'sony',
              'playstation_portable': 'sony',
              'xbox_360': 'microsoft',
              'wii': 'nintendo',
              'playstation_3': 'sony',
              'ds': 'nintendo',
              'playstation_2': 'sony'}

game['brand'] = game['platform'].map(brand_dict)

In [21]:
# create a mobile column

# build a dictionary to map from the platform column
mobile_dict = {'pc-fx': 0,
               'game_gear': 1,
               'turbo_grafx-16': 0,
               '3do': 0,
               'wonderswan': 1,
               'sega_cd': 0,
               'neo_geo': 0,
               'genesis': 0,
               'dreamcast': 0,
               'gameboy': 1,
               'nintendo_entertainment_system': 0,
               '2600': 0,
               'wii_u': 1,
               'saturn': 0,
               'super_nintendo_entertainment_system': 0,
               'xbox_one': 0,
               'nintendo_64': 0,
               'playstation_4': 0,
               'playstation_vita': 1,
               '3ds': 1,
               'game_cube': 0,
               'gameboy_advanced': 1,
               'xbox': 0,
               'computer': 0,
               'playstation': 0,
               'playstation_portable': 1,
               'xbox_360': 0,
               'wii': 0,
               'playstation_3': 0,
               'ds': 1,
               'playstation_2': 0}

game['mobile'] = game['platform'].map(mobile_dict)

#### numeric columns
___
release year, fill missing values<br>
user_score, update scale to match critic_score<br>
critic_score, replace placeholders<br>
sales
feature engineering, create total_sales column

In [22]:
# user_score, needs to be converted to float, it contains strings we should convert to missing
game['user_score'] = pd.to_numeric(game['user_score'], errors='coerce')

In [23]:
game.describe()

,year_of_release,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,mobile
count,16440.0000,16709.0000,16709.0000,16709.0000,16709.0000,8133.0000,7586.0000,16709.0000
mean,2006.4857,0.2632,0.1450,0.0776,0.0473,68.9658,7.1253,0.3222
std,5.8759,0.8135,0.5034,0.3089,0.1867,13.9361,1.4997,0.4673
min,1980.0000,0.0000,0.0000,0.0000,0.0000,13.0000,0.0000,0.0000
25%,2003.0000,0.0000,0.0000,0.0000,0.0000,60.0000,6.4000,0.0000
50%,2007.0000,0.0800,0.0200,0.0000,0.0100,71.0000,7.5000,0.0000
75%,2010.0000,0.2400,0.1100,0.0400,0.0300,79.0000,8.2000,1.0000
max,2016.0000,41.3600,28.9600,10.2200,10.5700,98.0000,9.7000,1.0000


year of release: <br>
mean < median, tail to the left (older) <br>
269 missing<br>
categorical: fill with 'null'<br>

critic score: <br>
lots of missing<br>

user_scoer:<br>
different scale than critic score but approximately same sample size

In [24]:
# make user score 1-100 like critic score
game['user_score'] = game['user_score'] * 10

In [25]:
# what percent of data has a missing year
miss_year = game[game['year_of_release'].isna()]

sales_total = game['na_sales'].sum() + game['eu_sales'].sum() + \
    game['jp_sales'].sum() + game['other_sales'].sum()

missing_sales = miss_year['na_sales'].sum() + miss_year['eu_sales'].sum() + \
    miss_year['jp_sales'].sum() + miss_year['other_sales'].sum()

print(missing_sales / sales_total)
print(len(miss_year))
len(miss_year)/len(game)

0.0111049865790422
269


0.016099108265006882

If this was sales data I would drop it. It is a categorical value so I will just fill missing with null. using the most frequent doesn't make a lot of sense here

In [26]:
# fill missing years
game['year_of_release'] = game['year_of_release'].fillna('null')

In [27]:
# create a new column for total sales
game['total_sales'] = game['na_sales'] + \
    game['eu_sales'] + game['jp_sales'] + game['other_sales']

In [28]:
game.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16709 entries, 0 to 16714
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16709 non-null  object 
 1   platform         16709 non-null  object 
 2   year_of_release  16709 non-null  object 
 3   genre            16709 non-null  object 
 4   na_sales         16709 non-null  float64
 5   eu_sales         16709 non-null  float64
 6   jp_sales         16709 non-null  float64
 7   other_sales      16709 non-null  float64
 8   critic_score     8133 non-null   float64
 9   user_score       7586 non-null   float64
 10  rating           16709 non-null  object 
 11  brand            16709 non-null  object 
 12  mobile           16709 non-null  int64  
 13  total_sales      16709 non-null  float64
dtypes: float64(7), int64(1), object(6)
memory usage: 1.9+ MB


In [29]:
# save csv
game.to_csv('../processed_games.csv', index=False)